In [1]:
import pandas as pd

task1_data_7b = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_finetune_data/sens_simp_7b_ft.csv")
# task2_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/bd_diff_70b_inf.csv")
print(task1_data_7b.shape)
# print(task2_data.shape)

(200, 4)


In [2]:
### Task 1
task1_data_7b.head()

,prompt,output,answer,ground_truth
0,<s>[INST] <<SYS>>Think step by step and decide...,Resistant.\nThe drug SMILES structure indicate...,Resistant,Sensitive
1,<s>[INST] <<SYS>>Think step by step and decide...,Resistant.\nThe drug SMILES structure indicate...,Resistant,Resistant
2,<s>[INST] <<SYS>>Think step by step and decide...,Resistant.\nThe drug SMILES structure indicate...,Resistant,Resistant
3,<s>[INST] <<SYS>>Think step by step and decide...,Resistant.\nThe drug sensitivity of the drug o...,Resistant,Sensitive
4,<s>[INST] <<SYS>>Think step by step and decide...,Resistant.\nThe drug SMILES structure indicate...,Resistant,Sensitive


In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
accuracy_score(task1_data_7b['ground_truth'], task1_data_7b['answer'])

0.475

In [36]:
confusion_matrix(task1_data_7b['ground_truth'], task1_data_7b['answer'], labels =  ["Sensitive", "Resistant"])

array([[  0, 105],
       [  0,  95]])

In [38]:
precision_score(task1_data_7b['ground_truth'], task1_data_7b['answer'], pos_label =  "Sensitive")

/home/yingfei/llm_data/llm_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.0

In [39]:
recall_score(task1_data_7b['ground_truth'], task1_data_7b['answer'], pos_label =  "Sensitive")

0.0

In [40]:
f1_score(task1_data_7b['ground_truth'], task1_data_7b['answer'], pos_label =  "Sensitive")

0.0

In [4]:
confusion_matrix(task1_data_7b['ground_truth'], task1_data_7b['answer'])

array([[ 95,   0],
       [105,   0]])

In [5]:
task1_data_70b = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/sens_simp_70b_inf_test.csv")
print(task1_data_70b.shape)

(200, 4)


In [6]:
### Task 1
task1_data_70b.head()

,cell_id,prompt,answer,output70
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, I would det..."
1,ACH-000006,Think step by step and decide in a single word...,Resistant,"Based on the information provided, my answer f..."
2,ACH-000006,Think step by step and decide in a single word...,Resistant,"Based on the information provided, I would det..."
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,Sensitive.\n\nThe drug is a tyrosine kinase in...
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, my answer f..."


In [7]:
task1_data_70b.loc[task1_data_70b.output70.apply(lambda x:len(x) < 10)]

,cell_id,prompt,answer,output70
12,ACH-000006,Think step by step and decide in a single word...,Resistant,Sensitive
18,ACH-000006,Think step by step and decide in a single word...,Resistant,Sensitive
24,ACH-000006,Think step by step and decide in a single word...,Resistant,Sensitive
35,ACH-000021,Think step by step and decide in a single word...,Sensitive,Resistant
48,ACH-000021,Think step by step and decide in a single word...,Resistant,Sensitive
52,ACH-000021,Think step by step and decide in a single word...,Resistant,Sensitive
110,ACH-000067,Think step by step and decide in a single word...,Resistant,Sensitive
142,ACH-000078,Think step by step and decide in a single word...,Resistant,Sensitive
146,ACH-000078,Think step by step and decide in a single word...,Resistant,Sensitive
168,ACH-000105,Think step by step and decide in a single word...,Resistant,Sensitive


In [8]:
### Validate the two should be on the same evaluation dataset
pd.Series(task1_data_7b.ground_truth == task1_data_70b.answer).value_counts()

True    200
Name: count, dtype: int64

In [9]:
for i in range(5):
    print(i, task1_data_70b['output70'][i])

0 Based on the information provided, I would determine the drug sensitivity of JW-7-24-1 on the cell line as Resistant.

The drug targets LCK, which is a kinase that plays a role in the signaling pathways of T cells. The cell line has mutations in several genes that are involved in kinase signaling, including FLT3, MAP3K1, and NOTCH1. These mutations may confer resistance to the drug by altering the signaling pathways that LCK regulates. Additionally, the cell line has a mutation in TP53, which is a tumor suppressor gene that can regulate cell death and proliferation. Mutations in TP53 can also contribute to drug resistance.

Therefore, based on the available information, it is likely that the cell line is resistant to JW-7-24-1.
1 Based on the information provided, my answer for the drug sensitivity of NVP-TAE684 on the cell line with the given mutations would be:

Sensitive
2 Based on the information provided, I would determine the drug sensitivity of KIN001-244 on the cell line with

In [10]:
task1_data_70b['prediction'] = task1_data_70b['output70'].apply(lambda x: "Check" if "sensitive" in x.lower() and "resistant" in x.lower() else \
                                                               "Sensitive" if "sensitive" in x.lower() else "Resistant" if "resistant" in x.lower() \
                                                               else "None")
task1_data_70b['prediction'].value_counts()

prediction
Sensitive    107
Resistant     86
Check          5
None           2
Name: count, dtype: int64

In [11]:
check_df = task1_data_70b.loc[task1_data_70b.prediction.isin(["Check", "None"])]
for i in range(len(check_df)):
    print(check_df.index.values[i])
    print(check_df.output70.values[i], check_df.prediction.values[i])
    print()

83
Based on the information provided, I would determine the drug sensitivity of BLEOMYCIN on the cell line with the given mutations as Resistant.

The drug target of BLEOMYCIN is dsDNA break induction, which is not directly related to the mutations present in the cell line. The mutations in the cell line are ARFRP1, IRS2, NBN, and WHSC1L1, which are not known to play a direct role in DNA replication or repair. Therefore, it is unlikely that the cell line would be sensitive to BLEOMYCIN, which works by inducing DNA double-strand breaks.

Hence, the drug sensitivity of BLEOMYCIN on the cell line with the given mutations would be Resistant. Check

84
Based on the information provided, I would determine the drug sensitivity of YM-201636 on the cell line with the given mutations as Resistant.

The drug targets PIKFYVE, which is involved in the PI3K/MTOR signaling pathway. The cell line has mutations in genes such as IRS2 and NBN, which are also involved in this pathway. These mutations may 

In [12]:
check_df

,cell_id,prompt,answer,output70,prediction
83,ACH-000045,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, I would det...",Check
84,ACH-000045,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, I would det...",Check
93,ACH-000045,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, the drug se...",None
123,ACH-000067,Think step by step and decide in a single word...,Resistant,"Based on the information provided, I would det...",Check
150,ACH-000105,Think step by step and decide in a single word...,Resistant,"Based on the information provided, I would det...",Check
182,ACH-000105,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, the drug se...",None
191,ACH-000107,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, the drug se...",Check


In [13]:
task1_data_70b.loc[task1_data_70b.prediction == "Check", 'prediction'] = "Resistant"
task1_data_70b.loc[task1_data_70b.prediction == "None", 'prediction'] = "Unknown"

In [14]:
task1_data_70b.prediction.value_counts()

prediction
Sensitive    107
Resistant     91
Unknown        2
Name: count, dtype: int64

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(task1_data_70b.answer, task1_data_70b.prediction)

array([[44, 51,  0],
       [47, 56,  2],
       [ 0,  0,  0]])

In [48]:
accuracy_score(task1_data_70b.answer, task1_data_70b.prediction)

0.5

In [45]:
precision_score(task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.5233644859813084

In [46]:
recall_score(task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.5436893203883495

In [47]:
f1_score(task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data_70b.loc[task1_data_70b.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.5333333333333333

In [17]:
### (*) Unknown -> two predicted as unknown
task1_data_70b.loc[task1_data_70b.prediction == "Unknown"]

,cell_id,prompt,answer,output70,prediction
93,ACH-000045,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, the drug se...",Unknown
182,ACH-000105,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, the drug se...",Unknown


In [18]:
print(task1_data_70b.loc[93, 'prompt'])
print(task1_data_70b.loc[93, 'output70']) ### Reasoning: the drug target is not directly related to the given mutation

Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line with given mutations: [Sensitive/Resistant], [Reasoning].
Drug and cell line mutations: 
The drug is TELOMERASE INHIBITOR IX. The drug SMILES structure is C1=CC(=CC(=C1)NC(=O)C2=C(C(=CC=C2)O)O)NC(=O)C3=C(C(=CC=C3)O)O. Drug target is Telomerase. Drug target pathway is Genome integrity.
The mutations of the cell line are ARFRP1, IRS2, NBN, WHSC1L1.
Drug Sensitivity: ?
Based on the information provided, the drug sensitivity of TELOMERASE INHIBITOR IX on the cell line with the given mutations is unknown.

The drug targets telomerase, which is not directly related to the given mutations. Therefore, it is unclear whether the drug would be effective on this cell line. Further research or experimentation would be necessary to determine the drug sensitivity of TELOMERASE INHIBITOR IX on this cell line.

Therefore, the answer is:

Drug Sensitivity: Unknown


In [19]:
print(task1_data_70b.loc[182, 'prompt'])
print(task1_data_70b.loc[182, 'output70']) ### Reasoning: the drug target and target pathway are unknown

Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line with given mutations: [Sensitive/Resistant], [Reasoning].
Drug and cell line mutations: 
The drug is BRYOSTATIN-1. The drug SMILES structure is CCCC=CC=CC(=O)OC1C(=CC(=O)OC)CC2CC(OC(=O)CC(CC3CC(C(C(O3)(CC4CC(=CC(=O)OC)CC(O4)C=CC(C1(O2)O)(C)C)O)(C)C)OC(=O)C)O)C(C)O. Drug target is Unknown. Drug target pathway is Unknown.
The mutations of the cell line are ERBB4, GNAS, NOTCH1, SDHA, SPEN, WT1.
Drug Sensitivity: ?
Based on the information provided, the drug sensitivity of BRYOSTATIN-1 on the cell line with the given mutations is unknown.

The drug targets Unknown, and its target pathway is also Unknown. Therefore, it is not possible to determine whether the drug will be effective on the cell line with the given mutations.

It's important to note that drug sensitivity can be influenced by many factors, including the specific mutations present in the cell line, the expression levels o

In [20]:
### (*) Redundancy of the answer (similarity between itself)
print(task1_data_70b["prompt"][0])
print(task1_data_70b["answer"][0])
print(task1_data_70b["prediction"][0])
task1_data_70b['output70'][0]

Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line with given mutations: [Sensitive/Resistant], [Reasoning].
Drug and cell line mutations: 
The drug is JW-7-24-1. The drug SMILES structure is COC1=CC(=CC(=C1)C2=CC3=C4C(=CN=C3C=C2)C=CC(=O)N4C5=CC(=C(C=C5)N6CCNCC6)C(F)(F)F)OC. Drug target is LCK. Drug target pathway is Other, kinases.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?
Sensitive
Resistant


'Based on the information provided, I would determine the drug sensitivity of JW-7-24-1 on the cell line as Resistant.\n\nThe drug targets LCK, which is a kinase that plays a role in the signaling pathways of T cells. The cell line has mutations in several genes that are involved in kinase signaling, including FLT3, MAP3K1, and NOTCH1. These mutations may confer resistance to the drug by altering the signaling pathways that LCK regulates. Additionally, the cell line has a mutation in TP53, which is a tumor suppressor gene that can regulate cell death and proliferation. Mutations in TP53 can also contribute to drug resistance.\n\nTherefore, based on the available information, it is likely that the cell line is resistant to JW-7-24-1.'

In [21]:
for reasoning in task1_data_70b['output70'].values:
    if "smiles" in reasoning.lower():
        print(reasoning)

Based on the information provided, I would determine the drug sensitivity of JQ1- (+) as Resistant.

The drug JQ1- (+) has a SMILES structure that suggests it is a bromodomain inhibitor, specifically targeting the BET protein family. However, the cell line mutations provided do not include any mutations in the BET protein family or their downstream targets. Instead, the mutations are scattered across various genes, including ALK, AR, ASXL1, ATR, BRCA2, CSF1R, KDM5C, MLL, MUTYH, SMARCA4, and TP53.

Given that the drug target is Unknown and the drug target pathway is Unknown, it is difficult to determine whether the drug would be effective against the cell line. However, based on the lack of mutations in the BET protein family or their downstream targets, it is likely that the drug would not be effective against the cell line, and therefore, I would classify the drug sensitivity as Resistant.
Based on the information provided, I would determine the drug sensitivity of JNJ-38877605 on the

In [22]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')  # Download the tokenizer model if you haven't already

# Example paragraph
paragraph = task1_data_70b['output70'][0]

# Tokenize the paragraph into sentences
sentences = sent_tokenize(paragraph)

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Transform sentences to TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(sentences)

# Calculate pairwise cosine similarities (excluding self-similarities)
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mask self-similarities by filling diagonal with 0
np.fill_diagonal(similarity_matrix, 0)

# Calculate average similarity (excluding self-similarities)
average_similarity = np.mean(similarity_matrix[similarity_matrix > 0])

print(f"Average Sentence Similarity (excluding self-similarities): {average_similarity:.2f}")
print("Higher values indicate more redundancy within the paragraph.")


Average Sentence Similarity (excluding self-similarities): 0.18
Higher values indicate more redundancy within the paragraph.


[nltk_data] Downloading package punkt to /home/yingfei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
vectorizer = TfidfVectorizer()

def reason_sim(reasoning):
    sentences = sent_tokenize(reasoning)
    tfidf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    np.fill_diagonal(similarity_matrix, 0)
    average_similarity = np.mean(similarity_matrix[similarity_matrix > 0])
    return average_similarity

task1_data_70b['reason_sim'] = task1_data_70b['output70'].apply(reason_sim)
task1_data_70b['reason_sim'].describe()

/home/yingfei/llm_data/llm_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/yingfei/llm_data/llm_env/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


count    148.000000
mean       0.230831
std        0.050149
min        0.133783
25%        0.194899
50%        0.225092
75%        0.262592
max        0.438274
Name: reason_sim, dtype: float64

In [24]:
task1_data_70b.loc[task1_data_70b['reason_sim']> 0.262592, 'prediction'].value_counts()

prediction
Resistant    20
Sensitive    16
Unknown       1
Name: count, dtype: int64

In [25]:
task1_data_70b.loc[task1_data_70b['reason_sim']< 0.194899, 'prediction'].value_counts()

prediction
Resistant    24
Sensitive    13
Name: count, dtype: int64

In [26]:
task1_data_70b.loc[task1_data_70b['reason_sim']> 0.4]

,cell_id,prompt,answer,output70,prediction,reason_sim
44,ACH-000021,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, my answer f...",Sensitive,0.438274
91,ACH-000045,Think step by step and decide in a single word...,Resistant,"Based on the information provided, the drug se...",Sensitive,0.430026


In [27]:
task1_data_70b.loc[task1_data_70b['reason_sim']> 0.4].output70.values

array(['Based on the information provided, my answer for the drug sensitivity of YK-4-279 on the given cell line would be:\n\nSensitive\n\nThe drug targets RNA helicase A, which is not listed as a mutated gene in the cell line. Therefore, it is likely that the drug will be effective against the cell line, resulting in sensitive drug sensitivity.',
       "Based on the information provided, the drug sensitivity of QUIZARTINIB on the cell line with mutations in ARFRP1, IRS2, NBN, and WHSC1L1 would be Sensitive.\n\nThe drug targets the FLT3 receptor, which is mutated in the cell line, and the drug's mechanism of action is to inhibit the activity of this receptor. Therefore, it is likely that the drug will be effective in inhibiting the activity of the mutated FLT3 receptor in this cell line, leading to sensitive drug sensitivity."],
      dtype=object)

In [28]:
### (*) Similarity of the answer with prompt (similarity between reasoning and prompt)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

paragraph1 = task1_data_70b["prompt"][0].split("Drug and cell line mutations:")[1].split("Drug Sensitivity:")[0]
paragraph2 = task1_data_70b['output70'][0]

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the paragraphs
tfidf_matrix = vectorizer.fit_transform([paragraph1, paragraph2])

# Calculate the cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# The similarity score
similarity_score = similarity_matrix[0][0]

print(f"Similarity Score: {similarity_score}")

Similarity Score: 0.3755198065605341


In [29]:
def prompt_sim(row):
    paragraph1 = row["prompt"].split("Drug and cell line mutations:")[1].split("Drug Sensitivity:")[0]
    paragraph2 = row['output70']    
    if len(paragraph2) < 10:
        return np.nan
    tfidf_matrix = vectorizer.fit_transform([paragraph1, paragraph2])
    similarity_matrix = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    similarity_score = similarity_matrix[0][0]
    return similarity_score

task1_data_70b['prompt_sim'] = task1_data_70b.apply(prompt_sim, axis = 1)
task1_data_70b['prompt_sim'].describe()

count    189.000000
mean       0.428793
std        0.075744
min        0.286507
25%        0.362309
50%        0.429722
75%        0.483506
max        0.638938
Name: prompt_sim, dtype: float64

In [30]:
task1_data_70b.loc[task1_data_70b['prompt_sim']> 0.483506, 'prediction'].value_counts()

prediction
Resistant    29
Sensitive    19
Name: count, dtype: int64

In [31]:
task1_data_70b.loc[task1_data_70b['prompt_sim']< 0.362309, 'prediction'].value_counts()

prediction
Sensitive    32
Resistant    14
Unknown       1
Name: count, dtype: int64

In [32]:
task1_data_70b.loc[task1_data_70b['prompt_sim']> 0.6]

,cell_id,prompt,answer,output70,prediction,reason_sim,prompt_sim
164,ACH-000105,Think step by step and decide in a single word...,Resistant,"Based on the information provided, the drug se...",Sensitive,0.309114,0.607228
169,ACH-000105,Think step by step and decide in a single word...,Sensitive,"Based on the information provided, I would det...",Resistant,0.223264,0.638938


In [33]:
task1_data_70b.loc[task1_data_70b['prompt_sim']> 0.6, 'output70'].values

array(["Based on the information provided, the drug sensitivity of GEFITINIB on the cell line is Sensitive.\n\nThe drug targets EGFR, which is mutated in the cell line, and the drug's target pathway is EGFR signaling. The presence of the mutation in the EGFR gene suggests that the cell line is likely to be sensitive to the drug. Therefore, the drug sensitivity is labeled as Sensitive.",
       'Based on the information provided, I would determine the drug sensitivity of DMOG on the cell line as Resistant.\n\nThe drug target of DMOG is HIF-PH, which is involved in the metabolism pathway. The cell line has mutations in genes such as ERBB4, GNAS, NOTCH1, SDHA, SPEN, and WT1, which are not directly related to the HIF-PH pathway. Therefore, it is likely that the cell line is resistant to DMOG treatment.'],
      dtype=object)

In [34]:
task1_data_70b.shape

(200, 7)